In [1]:
# A large portion of this code comes from the tf-learn example page:
# https://github.com/tflearn/tflearn/blob/master/examples/images/residual_network_cifar10.py
#https://github.com/safreita1/Resnet-Emotion-Recognition/blob/master/emotion_recognition.py

import tflearn
import numpy as np
import glob
import os
import scipy.misc as scm
from tflearn.data_preprocessing import ImagePreprocessing

In [56]:
def get_label(path, size):
    label = int(path[-5])
    one_hot = np.zeros(size)
    one_hot[ label ] = 1.0
    one_hot[ one_hot==0 ] = 0.0
    return one_hot

def get_image(img_path): # [0,255] to [-1,1]
    try:
        img = scm.imread(img_path)
    except:
        return None
    img = img * 2. /255. - 1.
    img = img[..., ::-1]  # rgb to bgr
    return img

In [84]:
# # load data test
# labelled_data_path = "../Face_data/Faces_with_expression_label/dataset_128x128"
# labelled_data = sorted(glob.glob(os.path.join(labelled_data_path, "*.*")))

# n_data = len(labelled_data)
# n_train = int(n_data * 0.8)
# n_val = int(n_data * 0.1)
# n_test = n_data - n_train - n_val

# # Shuffle
# random_order = np.random.permutation(n_data)
# labelled_data = [labelled_data[i] for i in random_order[:]]

# X = labelled_data[:n_train]
# X_val = labelled_data[n_train:n_train+n_val]
# X_test = labelled_data[-n_test:]

# Y = [get_label(x, 7) for x in X]
# Y_val = [get_label(x_val, 7) for x_val in X_val]
# Y_test = [get_label(x_test, 7) for x_test in X_test]

# X = [get_image(x) for x in X]
# X_val = [get_image(x_val) for x_val in X_val]
# X_test = [get_image(x_test) for x_test in X_test]


In [4]:
# Real-time image preprocessing
img_prep = tflearn.ImagePreprocessing()
# Zero Center (With mean computed over the whole dataset)
img_prep.add_featurewise_zero_center()
# STD Normalization (With std computed over the whole dataset)
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
# Random flip an image
img_aug.add_random_flip_leftright()

n = 5
# Building Residual Network
net = tflearn.input_data(shape=[None, 128, 128, 3], data_preprocessing=img_prep, data_augmentation=img_aug)
net = tflearn.conv_2d(net, nb_filter=16, filter_size=3, regularizer='L2', weight_decay=0.0001)
net = tflearn.residual_block(net, n, 16)
net = tflearn.residual_block(net, 1, 32, downsample=True)
net = tflearn.residual_block(net, n-1, 32)
net = tflearn.residual_block(net, 1, 64, downsample=True)
net = tflearn.residual_block(net, n-1, 64)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)

# Regression
net = tflearn.fully_connected(net, 7, activation='softmax')
mom = tflearn.Momentum(learning_rate=0.1, lr_decay=0.0001, decay_step=32000, staircase=True, momentum=0.9)
net = tflearn.regression(net, optimizer=mom,
                         loss='categorical_crossentropy')
# Training
model = tflearn.DNN(net, checkpoint_path='./models/model_resnet_emotion_3',
                    max_checkpoints=20, tensorboard_verbose=0,
                    clip_gradients=0.)

#model.load('model_resnet_emotion_1-42000')

#model.fit(X, Y, n_epoch=500, snapshot_epoch=False, snapshot_step=500, validation_set=(X_val, Y_val),
#          show_metric=True, batch_size=64, shuffle=True, run_id='resnet_emotion_3')

# score = model.evaluate(X_test, Y_test)
# print('Test accuarcy: ', score)

# model.save('model_3.tfl')
#prediction = model.predict(predict_value)
#print prediction

In [5]:
# model.load('model_3.tfl')
# score = model.evaluate(np.array(X_test), np.array(Y_test))
# print('Test accuarcy: ', score)


In [6]:
# print(np.array(X_test).shape, np.array(Y_test).shape)

In [7]:
# print(model.predict([X_test[0]]))

In [109]:
# load data test
celeba_gen_path = "./survey_image"
eval_data = sorted(glob.glob(os.path.join(celeba_gen_path, "*.*")))

X_eval = [get_image(x) for x in eval_data]
Y_eval = [get_label(x, 7) for x in eval_data]

model.load('./models/model_3.tfl')
score = model.evaluate(np.array(X_eval), np.array(Y_eval), batch_size=16)
print('Test accuarcy: ', score)


INFO:tensorflow:Restoring parameters from /home/tim/Workspace/ML/GANs/Face/Evaluations/models/model_3.tfl
Test accuarcy:  [0.92857146263122559]


In [110]:
import tensorflow as tf

_y=model.predict(np.array(X_eval))  # predictions
y = np.array(Y_eval) # i think this is actual labels data

confusion = tf.confusion_matrix(y.argmax(axis=1), _y.argmax(axis=1), num_classes=7)
sess = tf.Session() 
with sess.as_default(): 
    print(confusion.eval())

[[2 0 0 0 0 0 0]
 [0 2 0 0 0 0 0]
 [0 0 2 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0]
 [0 0 0 0 0 2 0]
 [0 0 0 1 0 0 1]]


In [111]:
print(eval_data)
print(_y.argmax(axis=1))

['./survey_image/ce_0.png', './survey_image/ce_1.png', './survey_image/ce_2.png', './survey_image/ce_3.png', './survey_image/ce_4.png', './survey_image/ce_5.png', './survey_image/ce_6.png', './survey_image/sj_0.png', './survey_image/sj_1.png', './survey_image/sj_2.png', './survey_image/sj_3.png', './survey_image/sj_4.png', './survey_image/sj_5.png', './survey_image/sj_6.png']
[0 1 2 3 4 5 6 0 1 2 3 4 5 3]


In [113]:
print(["{0:0.3f}".format(i) for i in _y[0]])
print(["{0:0.3f}".format(i) for i in _y[1]])
print(["{0:0.3f}".format(i) for i in _y[2]])
print(["{0:0.3f}".format(i) for i in _y[3]])
print(["{0:0.3f}".format(i) for i in _y[4]])
print(["{0:0.3f}".format(i) for i in _y[5]])
print(["{0:0.3f}".format(i) for i in _y[6]])


['1.000', '0.000', '0.000', '0.000', '0.000', '0.000', '0.000']
['0.000', '1.000', '0.000', '0.000', '0.000', '0.000', '0.000']
['0.000', '0.000', '0.999', '0.000', '0.000', '0.000', '0.000']
['0.003', '0.000', '0.000', '0.995', '0.000', '0.000', '0.001']
['0.000', '0.000', '0.000', '0.000', '0.978', '0.022', '0.000']
['0.000', '0.000', '0.000', '0.000', '0.000', '1.000', '0.000']
['0.008', '0.007', '0.001', '0.033', '0.001', '0.005', '0.944']


In [42]:
# Read Celeba attribute
celeba_txt_path = "../Face_data/Celeba/list_attr_celeba.txt"

attributes = []
with open(celeba_txt_path, 'r') as f:
    f.readline()
    attribute_names = f.readline().strip().split(' ')
    for i, line in enumerate(f):
        fields = line.strip().replace('  ', ' ').split(' ')
        img_name = fields[0]
        if int(img_name[:6]) != i + 1:
            raise ValueError('Parse error.')
        attr_vec = np.array([int(x) for x in fields[1:]])
        attributes.append(attr_vec)
attributes = np.array(attributes)


In [26]:
print(attribute_names)

['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']


In [86]:
selected_attributes = ['Male']
attr_idxs = [attribute_names.index(attr) for attr in selected_attributes]

target_images = []
for i in range(len(attributes)):
    img_attr = attributes[i]
#     if img_attr[attr_idxs] = 1:
    if img_attr[attr_idxs] == 1:
        for j in range(7):
            target_images.append('./gen_celeba/{0:06d}_{1}.jpg'.format(i+1, j))
        
        
        
print(target_images[:10])

['./gen_celeba/000003_0.jpg', './gen_celeba/000003_1.jpg', './gen_celeba/000003_2.jpg', './gen_celeba/000003_3.jpg', './gen_celeba/000003_4.jpg', './gen_celeba/000003_5.jpg', './gen_celeba/000003_6.jpg', './gen_celeba/000007_0.jpg', './gen_celeba/000007_1.jpg', './gen_celeba/000007_2.jpg']


In [87]:
# load data test
import tensorflow as tf

X_eval = []
Y_eval = []

num_exist = 0
for x in target_images[:1000]:
    img = get_image(x)
    if img is not None:
        X_eval.append(img)
        Y_eval.append(get_label(x, 7))
        num_exist += 1
    
# X_eval = [get_image(x) for x in target_images[:10000]]
# Y_eval = [get_label(x, 7) for x in target_images[:10000]]

print(num_exist)

model.load('./models/model_3.tfl')
score = model.evaluate(np.array(X_eval), np.array(Y_eval), batch_size=16)
print('Test accuarcy: ', score)


_y=model.predict(np.array(X_eval))  # predictions
y = np.array(Y_eval) # i think this is actual labels data


cm = None
confusion = tf.confusion_matrix(y.argmax(axis=1), _y.argmax(axis=1), num_classes=7)
sess = tf.Session() 
with sess.as_default(): 
    cm = confusion.eval()
print(cm)

874
INFO:tensorflow:Restoring parameters from /home/tim/Workspace/ML/GANs/Face/Evaluations/models/model_3.tfl
Test accuarcy:  [0.52288329546729817]
[[ 64   0   0   2   0  58   1]
 [  1  86   0   0   0  38   0]
 [  7   5  53   0   1  58   1]
 [ 13  13   2  50   0  43   4]
 [  1   2   0   0  38  84   0]
 [  0   0   0   0   0 125   0]
 [  8  14   7   6   0  48  41]]


In [92]:
confidence = []
_y_max_idx = _y.argmax(axis=1)
for i in range(len(_y)):
    one_y = _y[i]
    if one_y[_y_max_idx[i]] < 0.5:
        print(one_y[_y_max_idx[i]], _y_max_idx[i])
    


0.385628 1
0.413329 4
0.377358 2
0.373349 2
0.39877 6
0.308541 5
0.483653 6
0.404785 0
0.470733 2
0.464441 5
0.488168 0
0.297861 0
0.444193 1
0.483892 0
0.364209 1
0.477801 1
0.362948 0
0.391077 2
0.461696 2
0.336327 3
0.43668 1
0.490398 5
0.467698 0
0.429879 3
0.413001 6
0.335297 3
0.356414 1
0.465064 2
0.43027 3
0.4909 0
0.49832 3
0.492518 1
0.488265 3
0.426736 6
0.407255 6
0.435047 1
0.441148 5
0.473779 5
0.472963 6
0.480445 3
0.356281 6
0.492422 5
0.339946 0
0.442005 1
0.433826 2
0.484023 5
0.406391 1
0.475152 5


In [93]:
cm = None
with sess.as_default(): 
    cm = confusion.eval()
print(cm)

[[ 64   0   0   2   0  58   1]
 [  1  86   0   0   0  38   0]
 [  7   5  53   0   1  58   1]
 [ 13  13   2  50   0  43   4]
 [  1   2   0   0  38  84   0]
 [  0   0   0   0   0 125   0]
 [  8  14   7   6   0  48  41]]


In [85]:
# Test set
model.load('./models/model_3.tfl')
score = model.evaluate(np.array(X_test), np.array(Y_test))
print('Test accuarcy: ', score)


_y=model.predict(np.array(X_test))  # predictions
y = np.array(Y_test) # i think this is actual labels data



confusion = tf.confusion_matrix(y.argmax(axis=1), _y.argmax(axis=1), num_classes=7)
sess = tf.Session() 
with sess.as_default(): 
    print(confusion.eval())

INFO:tensorflow:Restoring parameters from /home/tim/Workspace/ML/GANs/Face/Evaluations/models/model_3.tfl
Test accuarcy:  [0.98636363636363633]
[[61  0  0  0  0  0  0]
 [ 0 66  0  0  1  0  0]
 [ 0  0 69  0  1  0  0]
 [ 0  0  0 30  0  0  0]
 [ 0  0  0  0 72  2  0]
 [ 0  1  0  0  0 63  0]
 [ 0  0  0  1  0  0 73]]


In [ ]:
# Data in survey
